# Размещение баннеров

Представим, что международное круизное агентство __Carnival Cruise Line__ решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — __Foursquare__.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

This data set contains 2153471 users, 1143092 venues, 1021970 check-ins, 27098490 social connections, and 2809581 ratings that users assigned to venues; all extracted from the Foursquare application through the public API. All users information have been anonymized, i.e., users geolocations are also anonymized. Each user is represented by an id, and GeoSpatial location. The same for venues. The data are contained in five files, users.dat, venues.dat, checkins.dat, socialgraph.dat, and ratings.dat. More details about the contents and use of all these files follows.

Нас будет интересовать файл __checkins.dat__. 

In [56]:
import pandas as pd

In [58]:
with open('fsq/umn_foursquare_datasets/checkins.dat') as checkins:
    line = checkins.readline().strip().split('|')
    features = [x.strip() for x in line]
    n_feature = len(columns)    

In [59]:
with open('fsq/umn_foursquare_datasets/checkins.dat') as checkins:
    checkins.readline()
    data_check = []
    for line in checkins:
        item = map(lambda x: x.strip(), line.strip().split('|'))
        if len(item) == n_feature: data_check.append(item)            

In [60]:
print feature, len(data)

6 396634


In [61]:
data = pd.DataFrame(data_check)
data.columns = features

In [62]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916,5222,,,2012-04-21 17:39:01
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391,5222,,,2012-04-21 17:37:18
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840,5222,,,2012-04-21 17:42:58


Удаляем ненужные

In [64]:
data = data[(data.latitude != '')]
data.shape

(396634, 6)

In [65]:
data.to_csv('data.csv')

In [67]:
data.tail()

,id,user_id,venue_id,latitude,longitude,created_at
1021959,955561,626076,20073,40.8501002,-73.8662464,2012-04-13 09:56:48
1021960,955892,674797,2297,33.7489954,-84.3879824,2012-04-13 10:56:03
1021961,956377,845102,11195,42.7653662,-71.467566,2012-04-13 12:08:45
1021962,956119,1139114,29488,42.439479,-83.7438303,2012-04-13 11:36:44
1021964,956733,960666,60,42.331427,-83.0457538,2012-04-13 21:56:19


In [81]:
data.user_id.value_counts()[:5]

1326476    48
386648     47
467043     46
8622       45
304865     45
Name: user_id, dtype: int64

## Кластеризация

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом __MeanShift__, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в __sklearn user guide__. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [68]:
from sklearn.cluster import MeanShift

In [72]:
data_checkins = data[['latitude', 'longitude']]

In [73]:
data_sample = data_checkins.sample(100000)

In [74]:
model = MeanShift(bandwidth=0.1)

In [76]:
%%time
model.fit(data_sample)

CPU times: user 11min 5s, sys: 1.89 s, total: 11min 6s
Wall time: 11min 18s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [108]:
model.cluster_centers_

array([[  40.71753292,  -73.98912337],
       [  41.87812206,  -87.62980074],
       [  33.44848844, -112.07409138],
       ..., 
       [  18.4274454 ,  -67.1540698 ],
       [  -4.5585849 ,  105.4068079 ],
       [  20.8650351 , -105.4390862 ]])

In [118]:
print len(model.labels_)
print model.labels_[:5]

100000
[694   0 917   0  22]


In [113]:
prediction = model.predict(data_sample)

In [142]:
data_sample['label'] = prediction
data_sample.head()

,latitude,longitude,label
945268,37.4315734,-78.6568942,694
713443,40.715972,-74.001437,0
274328,33.3806716,-84.7996573,917
1015188,40.715972,-74.001437,0
66662,34.0900091,-118.3617443,22


In [121]:
prediction[:5]

array([694,   0, 917,   0,  22])

In [152]:
cluster_size = pd.DataFrame(data_sample.pivot_table(index = 'label', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['size']

In [153]:
from collections import Counter

In [154]:
Counter(prediction);

In [155]:
import numpy as np

In [156]:
labels_unique = np.unique(model.labels_)
labels = model.labels_

In [157]:
n_clusters_ = len(labels_unique)
print n_clusters_

3608


In [158]:
model.labels_

array([694,   0, 917, ...,  48,   5,  21])

In [159]:
cluster_centers = pd.DataFrame(model.cluster_centers_)
cluster_centers.columns = ['latitude', 'longitude']

cluster_centers = cluster_centers.join(cluster_size)
cluster_centers.tail()

,latitude,longitude,size
3603,38.155750,-121.691344,1
3604,34.935395,-82.558738,1
3605,18.427445,-67.154070,1
3606,-4.558585,105.406808,1
3607,20.865035,-105.439086,1


In [161]:
cluster_centers.to_csv('clusters.csv', index = None)
cluster_centers= cluster_centers[(cluster_centers['size'] > 15)]
cluster_centers.tail()

,latitude,longitude,size
765,39.250517,-77.193451,25
772,44.936546,-93.666239,24
803,41.577780,-73.437540,18
1094,41.178427,-74.239797,27
1144,39.726725,-75.296743,16


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:



33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

In [163]:
coor_office = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании - широта и долгота этого центра, записанные через пробел.

In [164]:
import scipy as scp

In [178]:
def distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

def min_distance(x1, y1):
    min_dist = None
    for x2,y2 in coor_office:
        dist = distance(x1,y1,x2,y2)
        if (min_dist == None) or (dist < min_dist):
            min_dist = dist
    return min_dist
        
        

In [181]:
cluster_centers['min_dist'] = map(min_distance, cluster_centers.latitude, cluster_centers.longitude)
cluster_centers.tail()

,latitude,longitude,size,min_dist
765,39.250517,-77.193451,25,13.744086
772,44.936546,-93.666239,24,23.272983
803,41.577780,-73.437540,18,17.153146
1094,41.178427,-74.239797,27,16.475321
1144,39.726725,-75.296743,16,14.742661


In [183]:
clusters = cluster_centers.sort('min_dist')[:20]

/home/stas/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [187]:
for i in clusters.values:
    print i[0], i[1]

-33.8652215865 151.208195613
52.3728834848 4.89240072727
51.5035078596 -0.126551523759
25.9134926049 -80.2747587098
33.811378292 -118.143621406
25.7868963741 -80.2143981129
33.8891961026 -118.064634688
26.0096479056 -80.2012187645
33.6442974756 -117.948737151
26.1337302632 -80.3483951579
33.918284297 -118.414806369
26.1097176949 -80.1595997986
33.8113807695 -117.894821253
34.0598978755 -118.249425325
33.6880114059 -117.841578417
26.2122430289 -80.2402029156
34.0326666844 -118.438312887
34.127880378 -118.119934322
34.0861965489 -117.979572642
34.1287908266 -118.35278693
